In [1]:
a = 1 + 1
# some other things
print(a)

2


In [2]:
from mxnet import nd
from time import time

start = time()
x = nd.random_uniform(shape = (2000, 2000))
y = nd.dot(x, x)
print('workloads are queued:\t%f sec' % (time() - start))
print(y)
print('workloads are finished:\t%f sec' % (time()- start))

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


workloads are queued:	0.000924 sec

[[ 501.15838623  508.29724121  495.65237427 ...,  492.8470459   492.69091797
   490.0480957 ]
 [ 508.81057739  507.18218994  495.17428589 ...,  503.10525513
   497.29315186  493.6791687 ]
 [ 489.565979    499.47015381  490.17721558 ...,  490.99945068
   488.05007935  483.2883606 ]
 ..., 
 [ 484.00189209  495.71789551  479.92141724 ...,  493.69952393
   478.89193726  487.20739746]
 [ 499.64932251  507.65093994  497.59381104 ...,  493.0473938   500.74511719
   495.82711792]
 [ 516.01428223  519.17150879  506.35400391 ...,  510.08877563  496.3560791
   495.42523193]]
<NDArray 2000x2000 @cpu(0)>
workloads are finished:	0.175488 sec


In [3]:
start = time()
y = nd.dot(x, x)
y.wait_to_read()
time() - start

0.14577317237854004

In [4]:
start = time()
y = nd.dot(x, x)
z = nd.dot(x, x)
nd.waitall()
time() - start

0.28803491592407227

In [5]:
start = time()
y = nd.dot(x, x)
y.asnumpy()
time() - start

0.15755462646484375

In [6]:
start = time()
y = nd.dot(x, x)
y.norm().asscalar()
time() - start

0.1486518383026123

In [7]:
start = time()

for i in range(10000):
    y = x + 1
    y.wait_to_read()

print('No lazy evaluation: %f sec' % (time() - start))

start = time()
for i in range(10000):
    y = x + 1
nd.waitall()

print('With lazy evaluation: %f sec' % (time() - start))

No lazy evaluation: 14.823574 sec
With lazy evaluation: 14.071006 sec


In [8]:
a = 1
b = 2
a + b

3

In [9]:
def get_data():
    start = time()
    batch_size = 1024
    for i in range(60):
        if i % 10 == 0:
            print('batch %d, time %f sec' % (i, time() - start))
        x = nd.ones((batch_size, 1024))
        y = nd.ones((batch_size,))
        yield x, y

In [10]:
from mxnet import gluon
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Dense(1024, activation = 'relu'), 
        nn.Dense(1024, activation = 'relu'), 
        nn.Dense(1)
    )
net.initialize()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {})
loss = gluon.loss.L2Loss()

In [11]:
import os
import subprocess

def get_mem():
    '''get memory usage in MB'''
    res = subprocess.check_output(['ps', 'u', '-p', str(os.getpid())])
    return int(str(res).split()[15]) / 1e3

In [12]:
for x, y in get_data():
    break
loss(y, net(x)).wait_to_read()

batch 0, time 0.000001 sec


In [13]:
mem = get_mem()

for x, y in get_data():
    loss(y, net(x)).wait_to_read()
nd.waitall()

print('Increased memory %f MB' % (get_mem() - mem))

batch 0, time 0.000003 sec
batch 10, time 0.471921 sec
batch 20, time 0.912144 sec
batch 30, time 1.352273 sec
batch 40, time 1.792202 sec
batch 50, time 2.232561 sec
Increased memory 16.464000 MB


In [14]:
mem = get_mem()

for x, y in get_data():
    loss(y, net(x))

nd.waitall()
print('Increased memory %f MB' % (get_mem() - mem))

batch 0, time 0.000004 sec
batch 10, time 0.006003 sec
batch 20, time 0.010781 sec
batch 30, time 0.015448 sec
batch 40, time 0.021017 sec
batch 50, time 0.025144 sec
Increased memory 229.624000 MB


In [15]:
from mxnet import autograd as ag

mem = get_mem()

total_loss = 0
for x, y in get_data():
    with ag.record():
        L = loss(y, net(x))
    total_loss += L.sum().asscalar()
    L.backward()
    trainer.step(x.shape[0])

nd.waitall()
print('Increased memory %f MB' % (get_mem() - mem))

batch 0, time 0.000003 sec
batch 10, time 1.196729 sec
batch 20, time 2.472066 sec
batch 30, time 3.753315 sec
batch 40, time 5.024145 sec
batch 50, time 6.297179 sec
Increased memory -56.560000 MB


In [16]:
from mxnet import autograd as ag

mem = get_mem()

total_loss = 0
for x, y in get_data():
    with ag.record():
        L = loss(y, net(x))
    L.backward()
    trainer.step(x.shape[0])

nd.waitall()
print('Increased memory %f MB' % (get_mem() - mem))

batch 0, time 0.000002 sec
batch 10, time 0.011746 sec
batch 20, time 0.021163 sec
batch 30, time 0.030420 sec
batch 40, time 0.039908 sec
batch 50, time 0.055676 sec
Increased memory 94.120000 MB
